El objetivo de esta fase es extraer las features manuales para entrenar nuestras primeras versionas de nuestro modelo. Las features que se manejarán serán "promedio_intensidad", "proporcion_tinta_superior", y "simetria_vertical"

Antes de entrenar nuestro modelo con las features, vamos a dividir nuestro train dataset en 2 conjuntos de datos, train y validation, esto lo haremos con la intención de evitar el overfitting en nuestros modelos.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

# Ruta de guardado
output_dir = "../data/processed"
os.makedirs(output_dir, exist_ok=True)

# Cargar mnist_train
df_train = pd.read_csv("../data/raw/mnist_train.csv")

# Separar etiquetas e imágenes
y = df_train.iloc[:, 0]  # columna 0 = etiquetas
X = df_train.iloc[:, 1:] # columnas 1: = pixeles

# Dividir en train (80%) y val (20%)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Guardar en data/processed
X_train.to_csv(f"{output_dir}/X_train.csv", index=False)
y_train.to_csv(f"{output_dir}/y_train.csv", index=False)
X_val.to_csv(f"{output_dir}/X_val.csv", index=False)
y_val.to_csv(f"{output_dir}/y_val.csv", index=False)

# Cargar y guardar el set de test completo
df_test = pd.read_csv("../data/raw/mnist_test.csv")
y_test = df_test.iloc[:, 0]
X_test = df_test.iloc[:, 1:]

X_test.to_csv(f"{output_dir}/X_test.csv", index=False)
y_test.to_csv(f"{output_dir}/y_test.csv", index=False)


Una vez que hemos dividido nuestro train dataset vamos a proceder a crear nuestra función para aplicarle nuestras features a cada imagen de nuestra data, en este caso las features están en el archivo python "extractor".

In [7]:
import numpy as np
import sys
sys.path.append("..") 
from features import extractor

def generar_features(df_imagenes):
    feature_list = []

    for _, row in df_imagenes.iterrows():
        img = row.values.reshape(28, 28)

        features = [
            extractor.intensidad_promedio(img),
            extractor.varianza_intensidad(img),
            extractor.densidad_superior(img),
            extractor.simetria_vertical(img),
            extractor.centro_masa_vertical(img)
        ]

        feature_list.append(features)

    columnas = [
        "intensidad_promedio",
        "varianza",
        "densidad_superior",
        "simetria_vertical",
        "centro_masa_vertical"
    ]
    
    return pd.DataFrame(feature_list, columns=columnas)


Una vez hemos asignado cada feature a su respectiva imagen, procedemos a crear un nuevo .csv con las features de nuestra train, val y test data 

In [10]:
X_train_img = pd.read_csv("../data/processed/X_train.csv")
X_train_features = generar_features(X_train_img)
X_train_features.to_csv(f"{output_dir}/X_train_features.csv", index=False)

X_val_img = pd.read_csv("../data/processed/X_val.csv")
X_val_features = generar_features(X_val_img)
X_val_features.to_csv(f"{output_dir}/X_val_features.csv", index=False)

X_test_img = pd.read_csv("../data/processed/X_test.csv")
X_test_features = generar_features(X_test_img)
X_test_features.to_csv(f"{output_dir}/X_test_features.csv", index=False)

In [9]:
print(X_train_features)

       intensidad_promedio      varianza  densidad_superior  \
0                58.900510  10245.857449           0.404089   
1                27.184949   5060.655845           0.475813   
2                19.929847   3459.363701           0.439296   
3                42.220663   7771.077583           0.499592   
4                27.457908   4952.454861           0.546477   
...                    ...           ...                ...   
47994            49.110969   8562.187941           0.504169   
47995            32.433673   5751.028764           0.497640   
47996            37.014031   6711.661793           0.555154   
47997            13.234694   2528.962776           0.524383   
47998            35.595663   6576.309726           0.454151   

       simetria_vertical  centro_masa_vertical  
0              45.811224             13.901035  
1              29.676020             13.655703  
2              26.752551             13.939648  
3              54.323980             14.437570 